<a href="https://colab.research.google.com/github/the-faisalahmed/Optimization/blob/main/Knights%2C_Knaves_and_Spies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

# Knights, Knaves and Spies

An island has three kinds of inhabitants: knights, who always tell the truth, knaves, who always lie, and spies, who can either tell the truth or lie. You encounter three people: A, B, and C, and you know for sure that one is a knight, one is a knave, and one is a spy. Each of the three knows the type of the other two.

A says, “I am a spy”.
B says, “I am a spy”.
C says, “B is a spy”.
What are the types of A, B, and C?

In [ ]:
!pip install pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus
import pandas as pd

# Model Formulation

In [ ]:
model = ConcreteModel()

## 1. **Sets**:
- $I = (1, 2, 3)$ where (1,2,3) represent the three given constraints
- $J = (A, B, C)$ where (A,B,C) represents the three inhabitants

In [ ]:
constraints = [1,2,3]
inhabitants = ['A', 'B', 'C']

## 2. **Decision Variables**:
- $X_{i} \quad \forall i \in I$, variables tracking the truth value of each constraint
- $Y_{j} \quad \forall i \in J$, variables tracking the the location of our Spy

In [ ]:
# Truth of constraints variables
model.X = Var(constraints, within = Boolean)
# Determining which Y_[i] is a spy
model.Y = Var(inhabitants, within = Boolean)

## 3. **Constraints**:

    a. A says, “I am a spy”.
    b. B says, “I am a spy”.
    c. C says, “B is a spy”.

In [ ]:
# A says, “I am a spy”.
model.con1 = Constraint(expr = model.X[1] == model.Y['A'])

# B says, “I am a spy”.
model.con2 = Constraint(expr = model.X[2] == model.Y['B'])

# C says, “B is a spy”.
model.con3 = Constraint(expr = model.X[3] == (1-model.Y['B']))



## 4. **Model**:

\begin{align}
  && \text{Minimize:} \quad \text{None} && \\
  \text{s.t.} && \\
  && X[1] = Y[A] \tag{a}\\
  && X[2] = Y[B] \tag{b}\\
  && X[3] = 1-Y[B] \tag{c}\\
  && X_{i}, Y_{j} \in \{0,1\} && \forall i \in I, \forall j \in J \tag{d}
\end{align}

Constraints (a) and (b) equate the X variable to the respective Y variable, meaning that if this statement were true, then that inhabitant would be the spy and both the X and Y values would equal 1, otherwise 0. Constraint (c) equates $X_{3}$ to the opposite of the value of $Y_{B}$, meaning either the statement is true and $X_{3} = 0, Y_{B} = 1$ or the statement is false and $X_{3} = 1, Y_{B} = 0$. Constraint (d) ensures that all these variables are binary.

Note: Because we are only searching for a solution that satisfies the given constraints, there exists no true objective function to minimize/maximize.

In [1]:
def SolveModel(model, solver):
  # Solve model
  opt = SolverFactory(solver)
  result = opt.solve(model)

  if (result.solver.status == SolverStatus.ok) and \
      (result.solver.termination_condition == TerminationCondition.optimal):
      # Do something when the solution in optimal and feasible
      print('Solution is Optimal')
  elif (result.solver.termination_condition == TerminationCondition.infeasible):
      # Do something when model in infeasible
      print('Solution is Infeasible')
  else:
          # Something else is wrong
      print("Solver Status:",  result.solver.status)

  # Solve time
  print('Solve Time: ', result.solver.wallclock_time)

In [27]:
SolveModel(model, 'cbc')

Solution is Optimal
Solve Time:  0.0


## 5. **Solution**:

$X_1 = 0, X_2 = 1, X_3 = 0$, indicating that statement 2 (B says, "I am a spy'") and statement 3 (C says, "B is a spy") are true.

$Y_A = 0, Y_B = 1, Y_C = \text{None}$, agreeing with our X variables above.

**$Y_A$ = 0 so A is our Knave, $Y_B$ = 1 so B is our Spy, leaving $Y_C$ as our Knight** (ignore the 'None' value for $Y_C$).

In [42]:
X = pd.Series(model.X.extract_values())
Y = pd.Series(model.Y.extract_values())

print(X,Y)

1    0.0
2    1.0
3    0.0
dtype: float64 A    0.0
B    1.0
C    NaN
dtype: float64
